In [1]:
from numpy import  cos, pi
import numpy as np
import matplotlib.pyplot as plt
from floquet_simulations.hamiltonians import CreateHFGeneral
from floquet_simulations.periodic_functions import Cosine
from scipy.special import  jv
from scipy.linalg import eigh
import pandas as pd
import os
from pathlib import Path
from floquet_simulations.plot_functions import PlotParams, PlotAbsRealImagHamiltonian, PlotRealHamiltonian
from floquet_simulations.generate_gradients import ComputeAValsFromRequiredGradients, GetAValsFromBesselXVals
PlotParams(fontsize=12)

In [2]:

# get the A vals to get the right gradient

for Ndiffs in [100, 50]:
    for ymin in [-0.2, -0.1, jv(0, 3.8316)]:


        # xzero = 2.4048
        gradients = np.linspace(-ymin, ymin, Ndiffs) # for linear
        ideal_ham = np.zeros((Ndiffs+1, Ndiffs+1)) + np.diag(gradients,-1)+np.diag(gradients,1)   
        ideal_ham_dir = Path().absolute().parent/"hamiltonian_csvs"/f"ideal_ham_N={Ndiffs+1},grad={gradients[1]-gradients[0]:.4f}.csv"
        ideal_ham = pd.DataFrame(np.real(ideal_ham))
        ideal_ham.to_csv(ideal_ham_dir, 
                    index=False, header=False)

        # print(gradients)
        # print("change in tunnelling", f"{gradients[1]-gradients[0]:.4f}")
        # gradients = np.array([ymin, -ymin]*(int(Ndiffs/2)))  # for SSH
        # gradients = np.array([0.4, -0.1, 0.1]*4) #for SSH3
        # gradients = np.array([0.9, 0.4]*int(Ndiffs/2)) # for SSH not negative, makes A's lower
        xvals = ComputeAValsFromRequiredGradients(gradients)

        for omega in [25, 2, 40, 5]:
            for drive_style in ["accumulative", "alternating"]:
                print(Ndiffs, ymin, omega, drive_style)
                eff_ham_dir = Path().absolute().parent/"hamiltonian_csvs"/f"H_N={Ndiffs+1},w={omega},{drive_style},grad={gradients[1]-gradients[0]:.4f}.csv"

                if not os.path.isfile(eff_ham_dir):
                    print("y")

                    A_vals = GetAValsFromBesselXVals(xvals, omega, addition_type=drive_style, constant_shift="zero centre") # get actual shaking values


                    #oscilating A_vals
                    # A_vals = np.array([i%2 for i in range(Ndiffs +1)])*10


                    _, HF = CreateHFGeneral(Ndiffs+1,
                                            [int(i) for i in list(np.linspace(0,Ndiffs,Ndiffs+1))],
                                            [Cosine]*(Ndiffs+1),
                                            [[i,omega,0,0] for i in A_vals], #a, omega, phi onsite
                                            2*pi/omega,
                                            0
                                            )
                    #offset onsites
                    # for i in range(N):
                #     HF[i,i]=0
                    
                    # for i in range(N-2):
                    #     HF[i, i+2] = 0
                    #     HF[i+2, i] = 0

                    #make real
                    if np.all(np.imag(HF))==0:
                        HF =np.real(HF)



                    eff_ham_pd = pd.DataFrame(HF)
                    eff_ham_pd.to_csv(eff_ham_dir, 
                                index=False, header=False)

100 -0.2 25 accumulative
100 -0.2 25 alternating
100 -0.2 2 accumulative
100 -0.2 2 alternating
100 -0.2 40 accumulative
100 -0.2 40 alternating
100 -0.2 5 accumulative
100 -0.2 5 alternating
100 -0.1 25 accumulative
100 -0.1 25 alternating
100 -0.1 2 accumulative
100 -0.1 2 alternating
100 -0.1 40 accumulative
100 -0.1 40 alternating
100 -0.1 5 accumulative
y
100 -0.1 5 alternating
y
100 -0.40275939344110157 25 accumulative
y
100 -0.40275939344110157 25 alternating
y
100 -0.40275939344110157 2 accumulative
y
100 -0.40275939344110157 2 alternating
y
100 -0.40275939344110157 40 accumulative
y
100 -0.40275939344110157 40 alternating
y
100 -0.40275939344110157 5 accumulative
y
100 -0.40275939344110157 5 alternating
y
50 -0.2 25 accumulative
y
50 -0.2 25 alternating
y
50 -0.2 2 accumulative
y
50 -0.2 2 alternating
y
50 -0.2 40 accumulative
y
50 -0.2 40 alternating
y
50 -0.2 5 accumulative
y
50 -0.2 5 alternating
y
50 -0.1 25 accumulative
y
50 -0.1 25 alternating
y
50 -0.1 2 accumulative
y


In [3]:
# N= len(A_vals)
# markersize = 15
# fig, ax = plt.subplots(figsize=(5,5))
# ax.plot(range(N), A_vals, '.', markersize=markersize)
# ax.set_ylabel(r"$A$", rotation = 0)
# ax.set_xlabel(r"$site i$")
# ax.set_xticks(np.arange(0,N,10))
# plt.show()


In [4]:
# PlotAbsRealImagHamiltonian(HF, figsize=(8,8))
print(f"{gradients[1]-gradients[0]:.4f}")

-0.0164


In [5]:
# PlotRealHamiltonian(HF, figsize=(7, 7))


In [6]:

# ymax = 0.43
# # plot gradient
# fig, ax = plt.subplots(figsize=(8,6))
# JNN = [np.real(HF[i,i+1]) for i in range(N-1)]
# ax.plot(range(N-1), JNN, '.', markersize=markersize, label = r"$J_i$")
# # plt.plot(range(N-1), -gradients, label=r"linear gradient")
# ax.set_xlabel("i")
# ax.set_ylabel(r"$J_{i, i+1}$")
# ax.set_ylim([-ymax, ymax])
# ax.set_xticks(np.arange(0,N-1,10))
# # plt.legend()
# plt.show()


# # plot gradient 2
# JNNN = [np.real(HF[i,i+2]) for i in range(N-2)]
# fig, ax = plt.subplots(figsize=(8,6))
# ax.plot(range(N-2), JNNN, '.', markersize=markersize)

# ax.set_ylabel(r"$J_{i, i+2}$")
# ax.set_ylim([-ymax, ymax])
# ax.set_xlabel("site i")
# ax.set_xticks(np.arange(0,N-1,10))
# plt.show()


# onsite = [np.real(HF[i,i]) for i in range(N)]
# fig, ax = plt.subplots(figsize=(8,6))
# ax.plot(range(N), onsite, '.', markersize=markersize)
# ax.set_ylabel(r"$H_{i, i}$")
# ax.set_ylim([-ymax, ymax])
# ax.set_xlabel("site i")
# ax.set_xticks(np.arange(0,N,10))
# plt.show()

# JNNN_spread = np.abs(np.max(JNNN)) + np.abs(np.min(JNNN))
# JNN_spread = np.abs(np.max(JNN)) + np.abs(np.min(JNN))

# print("JNN/JNNN:", "{:.3f}".format(JNN_spread/JNNN_spread), "\t JNN spread:", "{:.3f}".format(JNN_spread), 
#       "\t JNNN spread:", "{:.3f}.".format(JNNN_spread), "\t A_val max:", "{:.3f}".format(np.max(A_vals)))